In [7]:
#importing necessary packages
import pandas as pd
import recordlinkage
import pathlib

In [8]:
#Read cleaned data into dataframe
Data_folder: str = pathlib.Path('C:\BBBrothers\BBB BETTER AND BETTER FILES\Data\merged_file.csv')
record_data = pd.read_csv(Data_folder)
record_data

C:\Users\arcoo\AppData\Local\Temp\ipykernel_4508\2676414173.py:3: DtypeWarning: Columns (2,3,4,5,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  record_data = pd.read_csv(Data_folder)


,ID,firm_id,company_name,condensed_name,main,legal,address_1,address_2,city,state,zip,zip_4,phone,email,url
0,1,2.0,"Able Fence, Inc.",ablefenceinc,True,True,78 Acker St E,NaN,Saint Paul,MN,55117,5507,6512224355,NaN,http://www.albinchapel.com/
1,2,5.0,Albin Chapel,albinchapel,False,False,2200 Nicollet Ave,NaN,Minneapolis,MN,55404,3301,9529149410,office@albinchapel.com,NaN
2,3,5.0,Albin Funeral Chapel Inc,albinfuneralchapelinc,False,False,6855 Rowland Rd,NaN,Eden Prairie,MN,55344,3209,6128711418,jimalbinson@gmail.com,http://www.arthurwilliamsoptical.com/
3,4,5.0,"Albin Endeavor, Inc.",albinendeavorinc,True,True,PO Box 46147,NaN,Eden Prairie,MN,55344,2847,6122700491,NaN,NaN
4,5,7.0,Albrecht Company,albrechtcompany,True,False,2024 Blackberry Ln,NaN,Wayzata,MN,55391,2006,6516334510,mail@albrechtcompany.com,http://www.ablemovers.net
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309324,327934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7016621044,NaN,NaN
309325,327935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5072084001,NaN,NaN
309326,327936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5072084150,NaN,NaN
309327,327937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6125672270,NaN,NaN


In [23]:
#Setup indexer to make candiate pairs of records within the dataset. Block candiate pair generation on records that have
#valid condensed names. A data sample of 100000 is used for computing constraints. 
indexer = recordlinkage.Index()
data_sample = record_data
indexer.block("condensed_name")
indexer.block("address_1")
canidate_links = indexer.index(data_sample)
print(len(data_sample), len(canidate_links))

309329 370941


In [25]:
"""
Setup comparision methods used to determine potential records that match. Comparision methods can either be
Numeric, string-based, or exact matches. Specific comparrsion methods are a work in progress to determine which
features/methods are the best for generating accurate matches. 
"""
compare_cl = recordlinkage.Compare()
#compare_cl.string("condensed_name", "condensed_name",method ="levenshtein",threshold=0.75, label="condensed_name")
compare_cl.string(
    "company_name", "company_name", method="levenshtein", threshold=0.75, label="company_name"
)
compare_cl.string("address_1", "address_1", method="levenshtein", threshold=0.85, label="address_1")
compare_cl.exact("state", "state", label="state")
compare_cl.string(
    "email", "email", method="levenshtein", threshold=0.85, label="email"
)
compare_cl.string(
    "url", "url", label = "url"
)
# Computes the matching score of each record based on the candiate links generated and the comparision criteria. 
features = compare_cl.compute(canidate_links, data_sample)

In [26]:
features.head(10)

,,company_name,address_1,state,email,url
29,27,1.0,0.0,0,0.0,0.484848
220,219,0.0,1.0,1,0.0,0.000000
377,376,1.0,0.0,1,0.0,0.000000
563,180,0.0,1.0,1,0.0,0.419355
566,177,0.0,1.0,1,0.0,0.600000
624,581,1.0,0.0,0,0.0,0.586207
705,704,0.0,1.0,1,0.0,0.000000
771,770,0.0,1.0,1,0.0,0.483871
793,434,0.0,1.0,1,0.0,0.226667
1016,123,0.0,1.0,1,0.0,0.387097


In [27]:
features.describe()

,company_name,address_1,state,email,url
count,370941.000000,370941.000000,370941.000000,370941.000000,370941.000000
mean,0.586352,0.411987,0.527332,0.000464,0.067503
std,0.492488,0.492193,0.499253,0.021528,0.160080
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,1.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [28]:
#Finds the similarity score of overall records which is based on the sum of each feature being matched.
round(features.sum(axis=1)).value_counts().sort_index(ascending=False)

4.0        40
3.0     12021
2.0    186564
1.0    171933
0.0       383
dtype: int64

In [29]:
#Potential matches are extracted from the features based on an arbirtray score of 3.5.
matches = features[round(features.sum(axis=1)) >= 3.5]
matches

,,company_name,address_1,state,email,url
6220,6219,1.0,0.0,1,1.0,0.527778
13312,13311,0.0,1.0,1,1.0,0.636364
22268,22267,1.0,0.0,1,1.0,0.878788
24010,24009,1.0,0.0,1,1.0,0.548387
24855,24854,1.0,0.0,1,1.0,0.586207
25508,25507,1.0,0.0,1,1.0,0.500000
26427,26426,1.0,1.0,1,0.0,0.551724
29443,29442,1.0,1.0,1,0.0,0.566667
29900,29899,1.0,1.0,1,0.0,0.695652
34054,34053,1.0,1.0,1,0.0,0.571429


In [ ]:
#The pairs of potential records are given IDs to be used for creating the final output
matches = matches.reset_index(names=['ID1','ID2'])

In [31]:
#Returns the values of a specific column in a Dataframe
def getColumnValues(df, column_name: str) -> list[str]:
    return df[column_name]

In [32]:
#Retrieves records from a DataFrame based on the values in a specified ID column.
def getRecordsByID(df, ID_Column_header: str) -> list[str]:
    #Gets values from a specific column based on the column name
    column_values = getColumnValues(df, ID_Column_header)
    #Intializes a list based on all the column headers that were in the orginal record_data
    record_list = [record_data.columns.values.tolist()]
    #For every record in the column, find the corresponding record in the record_data and append its information
    #into the record_list
    for value in column_values:
        row = record_data[record_data['ID'] == value]
        record_list.append(row.values.tolist())
    return record_list


In [33]:
#Gets a list of record information from the record_data dataframe of the records that were in the matches dataframe
ID_list1 = getRecordsByID(matches,"ID1")
ID_list2 = getRecordsByID(matches,"ID2")

In [39]:
ID_list1

[['ID',
  'firm_id',
  'company_name',
  'condensed_name',
  'main',
  'legal',
  'address_1',
  'address_2',
  'city',
  'state',
  'zip',
  'zip_4',
  'phone',
  'email',
  'url'],
 [[6220,
   2001297.0,
   "Brian's TV & Tanning",
   'brianstvandtanning',
   True,
   True,
   '4020 Stirrup Creek Dr Ste 100',
   nan,
   'Durham',
   'NC',
   '27703',
   '8998',
   '7017461333',
   'danb@centralvalleycoop.com',
   'http://www.benstoolandiron.com']],
 [[13312,
   14002726.0,
   'Associates The',
   'associatesthe',
   True,
   True,
   '601 Carlson Pkwy Ste 1550',
   nan,
   'Hopkins',
   'MN',
   '55305',
   '5232',
   '7635593185',
   'support@ion-technolgies.com',
   nan]],
 [[22268,
   25002242.0,
   "Harry's Auto Supply Inc",
   'harrysautosupplyinc',
   False,
   False,
   '2197 Oakgreen Ave N',
   nan,
   'Stillwater',
   'MN',
   '55082',
   '1526',
   '6126977355',
   'store1533@theupsstore.com',
   'http://www.cleanmyfurnace.com']],
 [[24010,
   38000304.0,
   'J T Edu com Inc

In [34]:
#Converts record data from a list of records and converts it into a Dataframe
def flattenRecords(record_list) -> pd.DataFrame:
    flattened_data = [item for sublist in record_list[1:] for item in sublist]
    df=pd.DataFrame(flattened_data, columns= ['ID', 'firm_id', 'company_name', 'condensed_name', 
                                              'main', 'legal', 'address_1', 'address_2', 'city', 
                                              'state', 'zip', 'zip_4', 'phone', 'email', 'url'])
    df.drop(['condensed_name','main','legal','zip','zip_4'],inplace=True,axis=1)
    return df

In [35]:
#Uses the above function to convert the list of values from record_data using the IDs from the matches dataframe
df1 = flattenRecords(ID_list1)
df2 = flattenRecords(ID_list2)

In [36]:
df1

,ID,firm_id,company_name,address_1,address_2,city,state,phone,email,url
0,6220,2001297.0,Brian's TV & Tanning,4020 Stirrup Creek Dr Ste 100,NaN,Durham,NC,7017461333,danb@centralvalleycoop.com,http://www.benstoolandiron.com
1,13312,14002726.0,Associates The,601 Carlson Pkwy Ste 1550,NaN,Hopkins,MN,7635593185,support@ion-technolgies.com,NaN
2,22268,25002242.0,Harry's Auto Supply Inc,2197 Oakgreen Ave N,NaN,Stillwater,MN,6126977355,store1533@theupsstore.com,http://www.cleanmyfurnace.com
3,24010,38000304.0,J T Edu com Inc,1026 Central Ave NE Ste 10,NaN,Minneapolis,MN,8007251655,robin114@tc.umn.edu,http://www.emeraldlawns.com
4,24855,41002044.0,Owl Book Store,10400 185th St N,NaN,Marine On Saint Croix,MN,9524879929,info@rswofmn.com,http://www.eclectrieffect.com
5,25508,43000644.0,Blaine Security,NaN,NaN,NaN,NaN,9529223616,joe@allwaysplumbinginc.com,http://www.skinbyjane.com
6,26427,43001860.0,R & R Mini-Storage,401 11th Ave S Ste 300,NaN,Hopkins,MN,2187224567,albertinazanda@yahoo.com,http://www.newbornmed.org
7,29443,51000677.0,Bloomington R V & Mini Storage Inc,7455 France Ave S,NaN,Minneapolis,MN,3209804623,gwheeler@prosourcetech.com,NaN
8,29900,51001412.0,Premium Windows & Siding Inc,1045 Arcade St,NaN,Saint Paul,MN,8004585834,jeanne@elitemortgagelending.com,http://www.winona.edu
9,34054,96004981.0,"LBL Advisors, LLC",25778 Hwy 71 NE,NaN,Blackduck,MN,9528732234,mcgrath@supergloo.com,http://www.faribaultprintshop.com


In [37]:
df2

,ID,firm_id,company_name,address_1,address_2,city,state,phone,email,url
0,6219,2001294.0,Niels Service,2955 Lone Oak Circle,NaN,Saint Paul,MN,8003424254,garym@centralvalleycoop.com,http://www.builtbybens.com
1,13311,14002712.0,D H Roofing,2565 Highway 10 NE,NaN,Saint Paul,MN,6514637447,cathy.carlisle@shercohomebuilders.com,NaN
2,22267,25002242.0,B & B Vet Supply,1547 152nd Ave NE,NaN,Andover,MN,6516977355,lexington.stillwater@yahoo.com,http://www.lakesvet.net
3,24009,38000303.0,Bright Ideas of Rochester Inc,NaN,NaN,NaN,NaN,7018828191,cindyka@midco.net,http://www.trevisimpressions.com
4,24854,41002036.0,Otis Elevator Co,246 Kingsway Dr,NaN,Mankato,MN,9527071212,rswconstruction@comcast.net,http://www.cpvalet.com
5,25507,43000644.0,"Blaine Lock & Safe, Inc.",4301 68th Ave N,NaN,Brooklyn Center,MN,3202538292,richards.capital@gmail.com,http://www.yelldrain.com
6,26426,43001858.0,Roof Depot,3600 Holly Ln N # 100,NaN,Minneapolis,MN,6122966000,info@abode-construction.com,http://www.crashtoys.com
7,29442,51000676.0,Mastercraft Electronics Inc,2021 Oakmont Ter,NaN,Pompano Beach,FL,3202406598,gwheeler@pressuretech.com,NaN
8,29899,51001399.0,Straight Line Auto Body,NaN,NaN,NaN,NaN,6512750123,lisa@brainbold.com,http://www.mnsu.edu
9,34053,96004981.0,Lurie Besikof Lapidus Private Investment Banki...,PO Box 416,NaN,Blackduck,MN,9524455465,info@supergloo.com,http://www.FranciscanResources.com
